<a href="https://colab.research.google.com/github/gabrielpatzer/mtg-art-search/blob/main/Busca_de_cartas_de_magic_pela_arte_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [2]:
!pip install -U -q google-generativeai

In [3]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

### Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [4]:
from google.colab import userdata

API_KEY=userdata.get('geminiapikey')
genai.configure(api_key=API_KEY)


### Parse the arguments

In [6]:
model = 'gemini-1.5-pro-latest' # @param {isTemplate: true}
contents_b64 = '...' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19PTkxZX0hJR0gifSx7ImNhdGVnb3J5IjoiSEFSTV9DQVRFR09SWV9IQVRFX1NQRUVDSCIsInRocmVzaG9sZCI6IkJMT0NLX09OTFlfSElHSCJ9LHsiY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX1NFWFVBTExZX0VYUExJQ0lUIiwidGhyZXNob2xkIjoiQkxPQ0tfTk9ORSJ9LHsiY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0RBTkdFUk9VU19DT05URU5UIiwidGhyZXNob2xkIjoiQkxPQ0tfT05MWV9ISUdIIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

## Call the API

In [10]:
def construir_consulta(termos):
  dic = json.loads(termos)
  lista = []
  for valores in dic.values():
    lista.append(f"(art:{'+OR+art:'.join(valores)})")  # Adicionando termos com OR
  query = "+".join(lista) # Juntando as categorias
  consulta = "https://api.scryfall.com/cards/search?q="+query+"&unique=art&limit=5"
  return consulta

### inicia o chat

In [12]:
gemini = genai.GenerativeModel(model_name=model)
chat = gemini.start_chat(history=contents)
user_input = ''

debug = False

while user_input != 'fim':
  user_input = input("Aguardando descrição de uma imagem: ")
  if user_input != 'fim':
    print('Buscando termos.')
    response = chat.send_message(
      user_input,
      stream=stream)
    termos_de_busca = response.text.replace('json','').replace('`',"")
    if debug:
      print(termos_de_busca)
    print('Gerando a consulta.')
    url_consulta = construir_consulta(termos_de_busca)
    print('Buscando em Scryfall')
    if debug:
      print(url_consulta)

    resposta = requests.get(url_consulta)
    dados_json = resposta.json()
    cartas = dados_json['data']
    if len(cartas) < 1 or dados_json['object']=="error":
      display(Markdown("## Nenhum resultado para a pesquisa"))
    else:
      output = "|Carta|Imagem|\n|:----|----:|\n"
      for c in cartas:
        linha = "|["+c['name']+"]("+c['scryfall_uri']+")|![Arte de "+c['name']+"]("+c['image_uris']['art_crop']+")|\n"
        if debug:
          print(linha)
        output += linha
      if debug:
        print(output)
      display(Markdown(output))



Aguardando descrição de uma imagem: macacos e bananas
Buscando termos.
Gerando a consulta.
Buscando em Scryfall


|Carta|Imagem|
|:----|----:|
|[Kibo, Uktabi Prince](https://scryfall.com/card/j22/40/kibo-uktabi-prince?utm_source=api)|![Arte de Kibo, Uktabi Prince](https://cards.scryfall.io/art_crop/front/8/b/8b71345a-c3e8-4b35-beb7-6347e41d7626.jpg?1675644790)|
|[Kibo, Uktabi Prince](https://scryfall.com/card/j22/835/kibo-uktabi-prince?utm_source=api)|![Arte de Kibo, Uktabi Prince](https://cards.scryfall.io/art_crop/front/d/a/da105c2f-017e-4df0-b45e-a9482c02a29d.jpg?1675625431)|
|[Monkey-](https://scryfall.com/card/ust/115/monkey-?utm_source=api)|![Arte de Monkey-](https://cards.scryfall.io/art_crop/front/5/6/569f48f9-db8c-458d-b1c8-fed5fd844bfd.jpg?1572374080)|


Aguardando descrição de uma imagem: fim
